<a href="https://colab.research.google.com/github/liviya7678/Integrating-ML-with-classical-docking/blob/main/Molecular_docking_using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Molecular docking of HER2 breast cancer receptor and Tucatinib ligand


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content


drive  HER2_receptor_clean.pdb	sample_data  TUC_ligand.pdbqt


In [5]:
# System packages
!apt-get update -qq
!apt-get install -y -qq openbabel
!apt-get install -y -qq autodock-vina



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [10]:
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
!bash miniconda.sh -b -p /usr/local/miniconda



PREFIX=/usr/local/miniconda
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local/miniconda


In [11]:
!eval "$(/usr/local/miniconda/bin/conda shell.bash hook)" && conda install -c conda-forge pdbfixer -y


Jupyter detected...

CondaToSNonInteractiveError: Terms of Service have not been accepted for the following channels. Please accept or remove them before proceeding:
    - https://repo.anaconda.com/pkgs/main
    - https://repo.anaconda.com/pkgs/r

To accept these channels' Terms of Service, run the following commands:
    conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
    conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

For information on safely removing channels from your conda configuration,
please see the official documentation:

    https://www.anaconda.com/docs/tools/working-with-conda/channels



In [13]:
!pip install git+https://github.com/openmm/pdbfixer.git



  Cloning https://github.com/openmm/pdbfixer.git to /tmp/pip-req-build-fw6e3e_l
  Running command git clone --filter=blob:none --quiet https://github.com/openmm/pdbfixer.git /tmp/pip-req-build-fw6e3e_l
  Resolved https://github.com/openmm/pdbfixer.git to commit 0f701349b1d0b7c91d2ca0db78359274d0793400
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 54.5 MB/s eta 0:00:00
  Created wheel for pdbfixer: filename=pdbfixer-1.11.0-py3-none-any.whl size=681683 sha256=801613cf4f8589684c3a70dc233edb3816d7cb553d82a610fb2833a1dab1f371
  Stored in directory: /tmp/pip-ephem-wheel-cache-4q5yinfc/wheels/0b/90/9a/de8a860c8290c72a309b03127bbfdf72a3f9a09a7b7faf8882
Successfully built pdbfixer


In [14]:
import pdbfixer
print(" PDBFixer installed successfully!")


 PDBFixer installed successfully!


Receptor pre docking preparation steps

In [42]:
# Load Receptor
fixer = PDBFixer ( filename = "/content/HER2_receptor_clean.pdb")

# 1. Repair missing atoms
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()

# 2.Add polar hydrogens

fixer.addMissingHydrogens(pH=7.4)

# 3. Save repaired receptor
PDBFile.writeFile(fixer.topology, fixer.positions, open('HER2_receptor_fixed.pdb', 'w'))






In [43]:
# clean the pdb receptor file

input_path = "/content/HER2_receptor_fixed.pdb"
output_path = "/content/HER2_receptor_clean_for_pdbqt.pdb"

# Keep only relevant PDB lines: ATOM, HETATM, TER, END
with open(input_path, "r") as fin, open(output_path, "w") as fout:
    for line in fin:
        if line.startswith(("ATOM", "HETATM", "TER", "END")):
            fout.write(line)

print(" Cleaned receptor written to:", output_path)

# Check first few lines to confirm
!head -20 /content/HER2_receptor_clean_for_pdbqt.pdb
!ls -lh /content/HER2_receptor_clean_for_pdbqt.pdb




 Cleaned receptor written to: /content/HER2_receptor_clean_for_pdbqt.pdb
ATOM      1  N   ALA A   1     -15.684  18.524 -34.525  1.00  0.00           N  
ATOM      2  H   ALA A   1     -14.485  18.573 -34.468  1.00  0.00           H  
ATOM      3  H2  ALA A   1     -15.929  19.647 -34.175  1.00  0.00           H  
ATOM      4  H3  ALA A   1     -15.892  18.558 -35.712  1.00  0.00           H  
ATOM      5  CA  ALA A   1     -16.360  17.398 -33.805  1.00  0.00           C  
ATOM      6  HA  ALA A   1     -16.498  17.732 -32.666  1.00  0.00           H  
ATOM      7  C   ALA A   1     -15.700  16.074 -34.185  1.00  0.00           C  
ATOM      8  O   ALA A   1     -15.766  15.667 -35.344  1.00  0.00           O  
ATOM      9  CB  ALA A   1     -17.834  17.383 -34.128  1.00  0.00           C  
ATOM     10  HB1 ALA A   1     -18.141  17.134 -35.265  1.00  0.00           H  
ATOM     11  HB2 ALA A   1     -18.517  18.345 -33.907  1.00  0.00           H  
ATOM     12  HB3 ALA A   1     -18.4

In [52]:
!wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local


PREFIX=/usr/local
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local


In [53]:
# Add conda to PATH
import sys, os
os.environ['PATH'] = '/usr/local/bin:' + os.environ['PATH']

# Install RDKit from conda-forge
!conda install -c conda-forge rdkit -y


Jupyter detected...

CondaToSNonInteractiveError: Terms of Service have not been accepted for the following channels. Please accept or remove them before proceeding:
    - https://repo.anaconda.com/pkgs/main
    - https://repo.anaconda.com/pkgs/r

To accept these channels' Terms of Service, run the following commands:
    conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
    conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

For information on safely removing channels from your conda configuration,
please see the official documentation:

    https://www.anaconda.com/docs/tools/working-with-conda/channels



Add gasteiger charges and covert receptor to pdbqt

In [60]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Load receptor
receptor = Chem.MolFromPDBFile("/content/HER2_receptor_clean_for_pdbqt.pdb", removeHs=False)

# Compute gasteiger charges
AllChem.ComputeGasteigerCharges(receptor)

# Write to PDBQT-like format (for Vina)
with open("/content/HER2_receptor_rigid.pdbqt", "w") as f:
   f.write("REMARK  Gasteiger charges added by RDKit\n")
   conf = receptor.GetConformer()
   for atom in receptor.GetAtoms():
     pos = conf.GetAtomPosition(atom.GetIdx())
     charge = atom.GetProp('_GasteigerCharge')
     f.write(f"ATOM  {atom.GetIdx()+1:>5}  {atom.GetSymbol():<2}  MOL A{atom.GetIdx()+1:>4}    "
                f"{pos.x:8.3f}{pos.y:8.3f}{pos.z:8.3f}  1.00  0.00           {atom.GetSymbol():>2}  "
                f"charge={float(charge):.4f}\n")
     f.write("END\n")

print(" HER2 receptor with Gasteiger charges saved as → /content/HER2_receptor_rigid.pdbqt")

 HER2 receptor with Gasteiger charges saved as → /content/HER2_receptor_rigid.pdbqt


Ligand preparation pre docking

In [23]:
#  Prepare the ligand
!obabel /content/TUC_ligand.pdbqt -O /content/TUC_ligand_prepared.pdbqt --gen3d --partialcharge gasteiger

#  Verify the output file
!ls -l /content/TUC_ligand_prepared.pdbqt

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /content/TUC_ligand.pdbqt)

1 molecule converted
-rw-r--r-- 1 root root 4234 Oct 27 06:51 /content/TUC_ligand_prepared.pdbqt


Grid box parameters


In [62]:
# Define your known grid box parameters
center_x = -0.5
center_y = 7
center_z = -17

size_x = 60
size_y = 60
size_z = 60

print(" GRID BOX PARAMETERS ")
print(f"Center (x, y, z): {center_x}, {center_y}, {center_z}")
print(f"Box Size (x, y, z): {size_x}, {size_y}, {size_z}")

 GRID BOX PARAMETERS 
Center (x, y, z): -0.5, 7, -17
Box Size (x, y, z): 60, 60, 60


In [115]:
#Convert C,S,O,N,H,P autodock vina compatible formats

import re

in_path  = "/content/HER2_receptor_rigid.pdbqt"       #
out_path = "/content/HER2_receptor_vina_ready.pdbqt"

def safe_float(s, default=0.0):
    try:
        return float(s)
    except:
        return default


map_types = {
    "C":"C","CA":"A","A":"A","N":"N","NA":"NA",
    "O":"O","OA":"OA","S":"S","SA":"SA","H":"H","HD":"HD",
    "P":"P","F":"F","CL":"Cl","BR":"Br","I":"I",
    "ZN":"Zn","CA":"Ca","FE":"Fe"
}

fixed = 0
skipped = 0
with open(in_path, "r") as fin, open(out_path, "w") as fout:
    for raw in fin:
        line = raw.rstrip("\n")
        if not (line.startswith("ATOM") or line.startswith("HETATM")):
            fout.write(line + "\n")
            continue


        try:
            record = line[0:6].strip() or "ATOM"
            serial = int(line[6:11].strip()) if line[6:11].strip() else 0
            name   = line[12:16].strip() or "X"
            resname= line[17:20].strip() or "MOL"
            chain  = line[21].strip() or "A"
            resseq = int(line[22:26].strip()) if line[22:26].strip() else 1
            x = safe_float(line[30:38].strip())
            y = safe_float(line[38:46].strip())
            z = safe_float(line[46:54].strip())
            occ = safe_float(line[54:60].strip()) if line[54:60].strip() else 1.00
            bfac= safe_float(line[60:66].strip()) if line[60:66].strip() else 0.00
        except Exception:
            # fallback token parsing
            toks = line.split()
            try:
                # naive fallback positions
                serial = int(toks[1])
                name = toks[2]
                resname = toks[3]
                chain = toks[4] if len(toks)>4 else "A"
                resseq = int(toks[5]) if len(toks)>5 else 1
                x = safe_float(toks[6])
                y = safe_float(toks[7])
                z = safe_float(toks[8])
                occ = safe_float(toks[9]) if len(toks)>9 else 1.00
                bfac = safe_float(toks[10]) if len(toks)>10 else 0.00
            except Exception:
                fout.write(line + "\n")
                skipped += 1
                continue


        charge_val = 0.0
        m_charge = re.search(r"charge\s*=\s*([-+]?[0-9]*\.?[0-9]+)", line)
        if m_charge:
            charge_val = safe_float(m_charge.group(1))

        atype = ""
        m_type = re.search(r"([A-Za-z]{1,3})\s+charge\s*=", line)
        if m_type:
            atype = m_type.group(1)
        else:
            atype = line[76:78].strip()
            if not atype:
                parts = line.split()
                if "charge=" in line:
                    parts2 = [p for p in parts if "charge=" not in p]
                    if parts2:
                        atype = parts2[-1]
                else:
                    atype = name[0] if name else "C"

        atype_up = atype.upper()
        chosen = map_types.get(atype_up, None)
        if chosen is None:

            if atype_up and atype_up[0] in ("C","N","O","S","H","P","F","I"):
                chosen = atype_up[0]
            else:
                chosen = "C"
        atype_final = chosen
        atom_line = (
            f"{record:<6}"
            f"{serial:5d} "
            f"{name:<4}"
            f"{resname:>3} "
            f"{chain:1s}"
            f"{resseq:4d}"
            f"   "
            f"{x:8.3f}{y:8.3f}{z:8.3f}"
            f"{occ:6.2f}{bfac:6.2f}"
            f"      "
            f"{atype_final:>6s}"
            f" charge={charge_val:8.4f}"
        )
        fout.write(atom_line + "\n")
        fixed += 1

print(f"Done. Wrote {fixed} ATOM/HETATM lines to: {out_path}  (skipped {skipped} unparsed lines)")




Done. Wrote 4673 ATOM/HETATM lines to: /content/HER2_receptor_vina_ready.pdbqt  (skipped 0 unparsed lines)


In [111]:
!head -30 /content/HER2_receptor_vina_ready.pdbqt

REMARK  Gasteiger charges added by RDKit
ATOM      1 N   MOL A   1    -15.684  18.524 -34.525  1.00  0.00           N charge= -0.3476
END
ATOM      2 H   MOL A   2    -14.485  18.573 -34.468  1.00  0.00           H charge=  0.3447
END
ATOM      3 H   MOL A   3    -15.929  19.647 -34.175  1.00  0.00           H charge=  0.3447
END
ATOM      4 H   MOL A   4    -15.892  18.558 -35.712  1.00  0.00           H charge=  0.3447
END
ATOM      5 C   MOL A   5    -16.360  17.398 -33.805  1.00  0.00           C charge=  0.1610
END
ATOM      6 H   MOL A   6    -16.498  17.732 -32.666  1.00  0.00           H charge=  0.1065
END
ATOM      7 C   MOL A   7    -15.700  16.074 -34.185  1.00  0.00           C charge=  0.2806
END
ATOM      8 O   MOL A   8    -15.766  15.667 -35.344  1.00  0.00           O charge= -0.2679
END
ATOM      9 C   MOL A   9    -17.834  17.383 -34.128  1.00  0.00           C charge= -0.0031
END
ATOM     10 H   MOL A  10    -18.141  17.134 -35.265  1.00  0.00           H charge=  

Create configuration file for autodock vina

In [112]:

config_content = f"""
receptor = /content/HER2_receptor_vina_ready.pdbqt
ligand = /content/TUC_ligand.pdbqt

center_x = {center_x}
center_y = {center_y}
center_z = {center_z}

size_x = {size_x}
size_y = {size_y}
size_z = {size_z}

exhaustiveness = 8
num_modes = 9
energy_range = 4
out = /content/docked_output.pdbqt

"""

with open("config.txt", "w") as f:
  f.write(config_content)

print(" Config file created successfully:")
!cat config.txt

 Config file created successfully:

receptor = /content/HER2_receptor_vina_ready.pdbqt
ligand = /content/TUC_ligand.pdbqt

center_x = -0.5
center_y = 7
center_z = -17

size_x = 60
size_y = 60
size_z = 60

exhaustiveness = 8
num_modes = 9
energy_range = 4
out = /content/docked_output.pdbqt



Run Docking using config file

In [113]:
!vina --config config.txt





AutoDock Vina v1.2.3
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V